In [2]:
using ITensors
using Plots

In [3]:
# Create the spin-one sites
N =100
sites = siteinds("S=1",N)

# Define the Heisenberg Hamiltonian operator
os = OpSum()
for j=1:N-1
    os += "Sz",j,"Sz",j+1
    os += 0.5,"S+",j,"S-",j+1
    os += 0.5,"S-",j,"S+",j+1
    os -= "Sz",j 
end
H = MPO(os,sites)
nothing

In [4]:
# Create a random matrix product state in order to start the algorithm
psi0 = randomMPS(sites)

# Define some parameters to control the sweeping
maxdim = 100
cutoff = 1E-10

# DMRG algorithm
de, energy0, E, steps, step = 1, 0, zeros(0), zeros(0), 0

while de > 1e-2  # convergence criteria
    step += 1
    append!( steps, step)
    energy, psi = dmrg(H,psi0; nsweeps=1, maxdim, cutoff)
    psi0 = psi
    de = abs(energy - energy0)
    energy0 = energy
    println("Step\t$step\t->\tEnergy difference\t=\t$de")
    append!( E, energy )
end

After sweep 1 energy=-142.27580426297945  maxlinkdim=9 maxerr=3.52E-16 time=23.118
Step	1	->	Energy difference	=	142.27580426297945
After sweep 1 energy=-144.238559011404  maxlinkdim=57 maxerr=9.99E-11 time=1.084
Step	2	->	Energy difference	=	1.962754748424544
After sweep 1 energy=-144.36495719952015  maxlinkdim=73 maxerr=9.97E-11 time=2.737
Step	3	->	Energy difference	=	0.12639818811615555
After sweep 1 energy=-144.43603282830856  maxlinkdim=100 maxerr=1.90E-10 time=3.291
Step	4	->	Energy difference	=	0.07107562878840668
After sweep 1 energy=-144.48673103030737  maxlinkdim=100 maxerr=6.07E-10 time=4.164
Step	5	->	Energy difference	=	0.05069820199881292
After sweep 1 energy=-144.5276080285263  maxlinkdim=100 maxerr=4.77E-10 time=4.541
Step	6	->	Energy difference	=	0.04087699821891988
After sweep 1 energy=-144.56328445275733  maxlinkdim=100 maxerr=1.70E-09 time=5.663
Step	7	->	Energy difference	=	0.03567642423104189
After sweep 1 energy=-144.59146563947715  maxlinkdim=100 maxerr=5.06E-0

In [62]:
# Plot the resulting energy after each sweep
plot(steps, E,
    tickfontsize=40,
    legendfontsize=40,
    labelfontsize=40,
    linewidth=5,
    lc=:red
    )

plot!(steps, E,
    seriestype=:scatter,
    mc=:blue,
    ms=20, ma=5
    )

plot!(size=(2000,1500), legend=false, margin=10Plots.mm)
xlabel!("Number of sweeps")
ylabel!("Ground State Energy [eV]")
savefig("ground_state_energy.png")